In [ ]:
a = 1+2
print(a)


3


In [3]:
from fastbook import *

In [8]:
urls = search_images_ddg("bird photos",max_images=1)


URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>